# Dynamic decorator

> Overcoming the classic decorator's limitations

In the previous lesson we saw that the decorated object could not access the methods of the original object.

We could replicate every single interface member inside the decorator but that's impractical. What we can do instead is automate this process by means of a **dynamic decorator**.

In this scenario we're going to create a class that deals with writing files, and we'll add some logging capabilities. We will initialize the class with a file.

What we are going to do however is treat the class as if it were a file: our class should mimic the behavior of a file, and we will add the logging capabilities additionally. For simplicity, we will only implement a single file method ([`writelines`](https://www.w3schools.com/python/ref_file_writelines.asp)), but you can use your imagination for the rest. Our implementation will simply call the original `writelines` method and add some logging.

However, we still want our class to behave like a file, which means that we would have to implement every single file method. Instead, we will redirect all attribute requests to the underlying file within our class by overriding `__getattr__`, `__setattr__` and `__delattr__`.

In [1]:
class FileWithLogging:
    def __init__(self, file):
        self.file = file

    def writelines(self, strings):
        """Our implemented file method with logging"""
        self.file.writelines(strings)
        print(f'wrote {len(strings)} lines')

    def __getattr__(self, item):
        return getattr(self.__dict__['file'], item)

    def __setattr__(self, key, value):
        if key == 'file':
            self.__dict__[key] = value
        else:
            setattr(self.__dict__['file'], key, value)

    def __delattr__(self, item):
        #delattr(self.__dict__['file'], item)
        delattr(self.file, item)

    # def __iter__(self):
    #     return self.file.__iter__()

    # def __next__(self):
    #     return self.file.__next__()

Here's what's happening in the code above:
* `__getattr__` is the getter: we simply call `getattr` with the file object and the attribute we want to recover from it.
* `__setattr__` is the setter: here we actually want to check whether we're trying to change the handle of the file (thereby replacing the file itself) or we're trying to modify a specific attribute within the file.
* `__delattr__` is the deleter: we simply call `delattr` with the file object and the attribute we want to delete.
* Both `__iter__` and `__next__` are used to make our file both iterable and an iterator, which is required for files. However, we can access both of these methods from `__getattr__`, so it's not necessary to implement them.

By implementing these 3 methods, we can access every single attribute and method of our file, thus saving us a lot of work.

As an additional note, you might recall that `__dict__` is a special attribute that all Python objects contain. Accessing `self.__dict__['file']` is identical to accessing `self.file` in most cases, but since we're overriding the getter, setter and deleter, we have to be very careful with some specific side effects:
* `self.file = value` is syntactic sugar for `self.__setattr__('file', value)`. If we try to replace line 15 with the simplified `self.file = value`, we would cause infinite recursion because we'd keep calling `__setattr__` recursively. The current line bypasses calling `__setattr__`.
* The line 11 could be simplified with `getattr(self.file, item)` but only because we're initializing our objext with a file; if `self.file` did not exist, it would cause another infinite recursion scenario because Python would try to call `__getattr__('file')` if it could not find `self.file`, which would trigget another `getattr(self.file, item)`, and so on. `getattr(self.__dict__['file'], item)` bypasses calling `__getattr__`; if Python doesn't find `__dict__['file']` it would simply raise a KeyError.
* The issues above do not affect `__delattr__`, so we include a simplified version for easier reading.

Let's test our dynamic decorator:

In [ ]:
file = FileWithLogging(open('hello.txt', 'w'))
file.writelines(['hello', 'world']) # Our implementation of writelines with logging
file.write('testing') # method from the file itself
file.close() # another method from the file itself

wrote 2 lines


Running the code above will generate a `hello.txt` file with `helloworldtesting` as its contents.

Dynamic decorators are great for very easily building decorators over types with large APIs that you don't want to prox over yourself. However, keep in mind that they may cause a performance penalty due to all of the additional calls on top of the original ones from the original object.